In [3]:
import pandas as pd
import csv
import numpy as np

In [4]:
f = open('../Data/Preproceso/storms_final_preproceso.csv')
csv_f = csv.reader(f)
cabeceras = csv_f.next()

tabla_pre = dict()

for row in csv_f:
    ep = dict(zip(cabeceras, row))
    if row[11] in tabla_pre:
        tabla_pre[row[11]].append(ep)
    else:
        tabla_pre[row[11]] = [ep]

In [8]:
tabla_trans = []

for ep_id in tabla_pre:
    ep_trans = {'EPISODE_ID':  0,
                'BEGIN_DATE' : 0,
                'END_DATE' :   0,
                'LAT' :        0,
                'LON' :        0,
                'RANGE':       0,
                'LLUVIA':      0,
                'VIENTO':      0,
                'TORNADO':     0,
                'GRANIZO':     0,
                'NIEVE':       0,
                'T. ELECTRICA':0                 
    }
    
    episodio = tabla_pre[ep_id]
    latitudes, longitudes, radios = [], [], []
    
    for evento in episodio: 
        if ep_trans['BEGIN_DATE'] == 0:
            ep_trans['BEGIN_DATE'] = np.datetime64(evento['BEGIN_DATE_TIME'])
        elif ep_trans['BEGIN_DATE'] >= np.datetime64(evento['BEGIN_DATE_TIME']):
            ep_trans['BEGIN_DATE'] = np.datetime64(evento['BEGIN_DATE_TIME'])
            
        if ep_trans['END_DATE'] == 0:
            ep_trans['END_DATE'] = np.datetime64(evento['END_DATE_TIME'])
        elif ep_trans['END_DATE'] <= np.datetime64(evento['END_DATE_TIME']):
            ep_trans['END_DATE'] = np.datetime64(evento['END_DATE_TIME'])
        
        if evento['EVENT_TYPE'] == 'Heavy Rain' and ep_trans['LLUVIA'] == 0:
            ep_trans['LLUVIA'] = 1
        elif evento['EVENT_TYPE'] == 'Flood' and ep_trans['LLUVIA'] <= 1:
            ep_trans['LLUVIA'] = 2
        elif evento['EVENT_TYPE'] == 'Flash Flood' and ep_trans['LLUVIA'] <= 2:
            ep_trans['LLUVIA'] = 3
        elif evento['EVENT_TYPE'] == 'Strong Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 1
        elif evento['EVENT_TYPE'] == 'High Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 2
        elif evento['EVENT_TYPE'] == 'Thunderstorm Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 3
        elif evento['EVENT_TYPE'] == 'Tornado':
            ep_trans['TORNADO'] = int(evento['TOR_F_SCALE'][-1]) + 1 
        elif evento['EVENT_TYPE'] == 'Hail' and ep_trans['GRANIZO'] < evento['MAGNITUDE']:
            ep_trans['GRANIZO'] = evento['MAGNITUDE']
        elif evento['EVENT_TYPE'] == 'Winter Storm':
            ep_trans['NIEVE'] = 1
        elif evento['EVENT_TYPE'] == 'Heavy Snow':
            ep_trans['NIEVE'] = 1
        elif evento['EVENT_TYPE'] == 'Lightning':
            evento['T. ELECTRICA'] = 1
            
        ep_trans['EPISODE_ID'] = ep_id
       
        latitud = (float(evento['BEGIN_LAT']) + float(evento['END_LAT'])) / 2
        longitud = (float(evento['BEGIN_LON']) + float(evento['END_LON'])) / 2
        radio = float(evento['RANGE']) + dist(float(evento['BEGIN_LON']), float(evento['BEGIN_LAT']), longitud, latitud)
        
        latitudes.append(latitud)
        longitudes.append(longitud)
        radios.append(radio)
    
    
    centro = (np.average(longitudes), np.average(latitudes))
    
    distan = 0
    for lon, lat, r in zip(longitudes, latitudes, radios):
        d = dist(centro[0], centro[1], lon, lat) +  r
        if d > distan:
            distan = d
    
    ep_trans['LON'] = centro[0]
    ep_trans['LAT'] = centro[1]
    ep_trans['RANGE'] = distan
        
    tabla_trans.append(ep_trans)    

/home/pablo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/pablo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [7]:
import math

def merc_x(lon):
  r_major=6378137.000
  return r_major*math.radians(lon)

def merc_y(lat):
  if lat>89.5:lat=89.5
  if lat<-89.5:lat=-89.5
  r_major=6378137.000
  r_minor=6356752.3142
  temp=r_minor/r_major
  eccent=math.sqrt(1-temp**2)
  phi=math.radians(lat)
  sinphi=math.sin(phi)
  con=eccent*sinphi
  com=eccent/2
  con=((1.0-con)/(1.0+con))**com
  ts=math.tan((math.pi/2-phi)/2)/con
  y=0-r_major*math.log(ts)
  return y

#Distancia entre dos punto geograficos. Se obtiene sus proyecciones Mercator y la distancia euclidea entre ellas en km
def dist(p1lon,p1lat,p2lon,p2lat):
	x1=merc_x(p1lon)
	x2=merc_x(p2lon)
	y1=merc_y(p1lat)
	y2=merc_y(p2lat)
	return math.sqrt((x1-x2)**2+(y1-y2)**2) / 1000

In [10]:
df = pd.DataFrame(tabla_trans)
df = df.set_index('EPISODE_ID')
df.to_csv('../Data/Transformacion/transformacion.csv')

In [12]:
import matplotlib.pyplot as plt

plt.scatter(df['LON'], df['LAT'], s = df['RANGE'], alpha=0.7)
plt.show()

In [15]:
df['LON']

EPISODE_ID
73398     -86.214230
65098     -84.967010
4025      -86.030096
22667     -86.615000
22660     -86.491014
61456     -85.510000
14290     -86.241626
1211378   -87.695000
73088     -87.646838
1211371   -87.300000
1211372   -87.240000
32836     -86.316210
1211375   -87.550000
1211377   -87.885000
75542     -85.664211
65279     -87.319387
61195     -85.870148
61190     -86.220347
104447    -87.606600
25380     -85.887350
3772      -87.328571
3771      -85.900000
3770      -85.900000
1184414   -86.100000
1184413   -85.985000
32014     -86.650825
1219870   -85.182500
41833     -87.624950
1201292   -86.100000
1201296   -86.110000
             ...    
67358     -85.018375
52504     -86.269300
44897     -85.526568
44892     -86.125925
44893     -85.979525
50733     -85.090104
95319     -86.744100
45        -85.175000
40        -86.326000
1569      -87.660000
30084     -86.460984
30082     -86.628925
30083     -86.900000
18067     -86.880000
15270     -86.780000
61673     -85.729975
85